# <center>Detroit Lions Offensive Line Analysis</center>

<img src='https://static.clubs.nfl.com/image/upload/t_editorial_landscape_12_desktop/lions/p3q8ujlelurs5m2uu1kt' style='width:800px;margin:auto'/>

In [1]:
#import libraries and datasets
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/games.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/players.csv')
player_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/player_play.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/plays.csv')
week4 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv')

/kaggle/input/nfl-big-data-bowl-2025/players.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv
/kaggle/input/nfl-big-data-bowl-2025/games.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv
/kaggle/input/nfl-big-data-bowl-2025/player_play.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv
/kaggle/input/nfl-big-data-bowl-2025/plays.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv


## <center>Overview</center>

The main objective of this notebook is to analyze how the offensive linemen can help us predict whether the play is going to be a run or a pass play. We will need to overlap this with formation of the offense to see more valuable insights (an empty formation is almost certainly a pass play)

In [2]:
det_sea = week4[week4['gameId'] == 2022100205]
det_sea_plays = plays[plays['gameId'] == 2022100205]
det_sea[det_sea['club'] == 'DET']['displayName'].unique()

array(['Michael Brockers', 'Charles Harris', 'Alex Anzalone',
       'DeShon Elliott', 'Amani Oruwariye', 'Jeff Okudah', 'Alim McNeill',
       'Derrick Barnes', 'Aidan Hutchinson', 'Kerby Joseph',
       'Malcolm Rodriguez', 'Mike Hughes', 'Isaiah Buggs',
       'Julian Okwara', 'Benito Jones', 'Chris Board', 'Jared Goff',
       'Taylor Decker', 'Josh Reynolds', 'Jamaal Williams', 'Dan Skipper',
       'Frank Ragnow', 'Evan Brown', 'T.J. Hockenson', 'Tom Kennedy',
       'Quintez Cephus', 'Penei Sewell', 'Kalif Raymond',
       'Justin Jackson', 'Brock Wright', 'Matt Nelson', 'Craig Reynolds',
       'Austin Bryant', 'Will Harris', 'James Mitchell',
       'Maurice Alexander', 'Kayode Awosika'], dtype=object)

In [3]:
pd.set_option('display.max_colwidth', None)
detroit_plays = det_sea_plays[det_sea_plays['possessionTeam'] == 'DET']
detroit_plays[detroit_plays['passResult'].isna()]['offenseFormation'].value_counts()

detroit_plays[detroit_plays['offenseFormation'] == 'SINGLEBACK']

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
5,2022100205,2314,(14:15) Ja.Williams up the middle to DET 32 for 3 yards (C.Barton; U.Nwosu).,3,2,6,DET,SEA,DET,29,...,3,-0.024210,0.024210,-0.442517,False,MAN,NaN,0,Cover 6-Left,Zone
201,2022100205,2625,(7:11) M.Nelson reported in as eligible. Ja.Williams right end to DET 49 for 3 yards (S.Jones; B.Mafe).,3,1,10,DET,SEA,DET,46,...,3,-0.012892,0.012892,-0.324285,False,MAN,NaN,0,Cover-1,Man
762,2022100205,1468,(9:02) J.Goff pass short left to Ja.Williams pushed ob at DET 17 for 1 yard (B.Mafe).,2,1,10,DET,SEA,DET,16,...,1,-0.022358,0.022358,-0.600837,True,OUTSIDE ZONE,NaN,0,Cover-3,Zone
1607,2022100205,1084,(15:00) C.Reynolds up the middle pushed ob at SEA 27 for 21 yards (T.Woolen).,2,2,10,DET,SEA,SEA,48,...,21,0.043709,-0.043709,1.705531,False,POWER,NaN,0,Quarters,Zone
2280,2022100205,1915,(2:56) J.Goff pass short left to Q.Cephus to SEA 46 for 9 yards (J.Brooks).,2,1,10,DET,SEA,DET,45,...,9,0.020759,-0.020759,0.813811,True,NaN,NaN,0,Cover-2,Zone
3210,2022100205,1133,(14:00) Ja.Williams right tackle to SEA 31 for 1 yard (J.Brooks; U.Nwosu).,2,1,15,DET,SEA,SEA,32,...,1,-0.010306,0.010306,-0.476627,False,COUNTER,NaN,0,Cover-6 Right,Zone
3350,2022100205,2377,"(12:34) J.Goff pass short right to J.Jackson to DET 38 for 2 yards (B.Mafe, C.Barton).",3,2,10,DET,SEA,DET,36,...,2,-0.025951,0.025951,-0.862718,True,NaN,NaN,0,Cover-3,Zone
5115,2022100205,2648,"(6:39) Ja.Williams up the middle for 51 yards, TOUCHDOWN.",3,2,7,DET,SEA,DET,49,...,51,0.063580,-0.063580,4.549468,False,POWER,NaN,0,Cover-2,Zone
6248,2022100205,443,"(8:16) Ja.Williams left tackle to DET 31 for 5 yards (Q.Diggs, C.Bryant).",1,1,10,DET,SEA,DET,26,...,5,0.001581,-0.001581,-0.036492,False,OUTSIDE ZONE,NaN,0,Quarters,Zone
6524,2022100205,1497,(8:32) Ja.Williams left end to DET 21 for 4 yards (J.Brooks).,2,2,9,DET,SEA,DET,17,...,4,-0.005084,0.005084,-0.210817,False,POWER,NaN,0,Cover-3,Zone


In [4]:
detroit_plays[detroit_plays['offenseFormation'] == 'SINGLEBACK']
run_sb = week4[(week4['playId'] == 2648) & (week4['gameId'] == 2022100205)]
pass_sb = week4[(week4['playId'] == 1570) & (week4['gameId'] == 2022100205)]

### <center>Run play</center>

In [5]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'



fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

fig_dict['layout']['xaxis'] = {
    'range': [0, 100],
    'title': {'text': 'x', 'font': {'color': 'white'}},  
    'linecolor': 'black',  
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': True 
}

fig_dict['layout']['yaxis'] = {
    'range': [0, 50],
    'title': {'text': 'y', 'font': {'color': 'white'}},
    'linecolor': 'black', 
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': False  
}

fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['font'] = {'color': 'white'} 
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                'args': [None, {'frame': {'duration': 50, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 1000}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20, 'color': 'black'},
        'prefix': 'Time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300},
    'pad': {'b': 10, 't': 50},
    'len': .9,
    'x': .1,
    'y': 0,
    'steps': []
}

for club in pass_sb['club'].unique():
    data_dict = {
        'x': pass_sb.loc[pass_sb['club'] == club, 'x'],
        'y': pass_sb.loc[pass_sb['club'] == club, 'y'],
        'mode': 'markers',
        'text': pass_sb.loc[pass_sb['club'] == club, 'displayName'],
        'name': club
    }
    fig_dict['data'].append(data_dict)

initial_frame = {'data': [], 'name': 'Initial'}
fig_dict['frames'].append(initial_frame)

for frame in pass_sb['frameId'].unique():
    f = {'data': [], 'name': str(frame)}
    
    for club in pass_sb['club'].unique():
        frame_data = {
            'x': pass_sb.loc[pass_sb['frameId'] == frame, 'x'][pass_sb['club'] == club],
            'y': pass_sb.loc[pass_sb['frameId'] == frame, 'y'][pass_sb['club'] == club],
            'mode': 'markers',
            'text': pass_sb.loc[pass_sb['frameId'] == frame, 'displayName'][pass_sb['club'] == club],
            'name': club
        }
        f['data'].append(frame_data) 

    fig_dict['frames'].append(f)
    
    slider_step = {
        'args': [
            [frame],
            {'frame': {'duration': 0, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 0}}
        ],
        'label': str(frame),
        'method': 'animate'
    }
    sliders_dict['steps'].append(slider_step) 

fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

if len(fig_dict['frames']) > 1:
    fig.update(data=fig_dict['frames'][1]['data']) 

fig.update_layout(paper_bgcolor='#242424', plot_bgcolor='#0a4200')
    
fig.show()

### <center>Pass play</center>

In [6]:
fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

fig_dict['layout']['xaxis'] = {
    'range': [0, 100],
    'title': {'text': 'x', 'font': {'color': 'white'}},  
    'linecolor': 'black',  
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': True 
}

fig_dict['layout']['yaxis'] = {
    'range': [0, 50],
    'title': {'text': 'y', 'font': {'color': 'white'}},
    'linecolor': 'black', 
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': False  
}

fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['font'] = {'color': 'white'} 
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                'args': [None, {'frame': {'duration': 50, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 1000}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20, 'color': 'black'},
        'prefix': 'Time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300},
    'pad': {'b': 10, 't': 50},
    'len': .9,
    'x': .1,
    'y': 0,
    'steps': []
}

for club in pass_sb['club'].unique():
    data_dict = {
        'x': pass_sb.loc[pass_sb['club'] == club, 'x'],
        'y': pass_sb.loc[pass_sb['club'] == club, 'y'],
        'mode': 'markers',
        'text': pass_sb.loc[pass_sb['club'] == club, 'displayName'],
        'name': club
    }
    fig_dict['data'].append(data_dict)

initial_frame = {'data': [], 'name': 'Initial'}
fig_dict['frames'].append(initial_frame)

for frame in pass_sb['frameId'].unique():
    f = {'data': [], 'name': str(frame)}
    
    for club in pass_sb['club'].unique():
        frame_data = {
            'x': pass_sb.loc[pass_sb['frameId'] == frame, 'x'][pass_sb['club'] == club],
            'y': pass_sb.loc[pass_sb['frameId'] == frame, 'y'][pass_sb['club'] == club],
            'mode': 'markers',
            'text': pass_sb.loc[pass_sb['frameId'] == frame, 'displayName'][pass_sb['club'] == club],
            'name': club
        }
        f['data'].append(frame_data) 

    fig_dict['frames'].append(f)
    
    slider_step = {
        'args': [
            [frame],
            {'frame': {'duration': 0, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 0}}
        ],
        'label': str(frame),
        'method': 'animate'
    }
    sliders_dict['steps'].append(slider_step) 

fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

if len(fig_dict['frames']) > 1:
    fig.update(data=fig_dict['frames'][1]['data']) 

fig.update_layout(paper_bgcolor='#242424', plot_bgcolor='#0a4200')
    
fig.show()

In [7]:
run_lineset = pd.DataFrame()

for player in run_sb['nflId'].unique().tolist()[:-1]:
    df_tobeadded = run_sb[run_sb['nflId'] == player]
    lineset_index = df_tobeadded.index[df_tobeadded['event'] == 'line_set'].tolist()
    lineset_index = lineset_index[0]
    ballsnap_index = df_tobeadded.index[df_tobeadded["event"] == "ball_snap"].tolist()
    ballsnap_index = ballsnap_index[0]

    df_tobeadded = df_tobeadded.loc[lineset_index:ballsnap_index]

    run_lineset = pd.concat([run_lineset, df_tobeadded])

ol_players = ['Penei Sewell', 'Evan Brown', 'Frank Ragnow', 'Dan Skipper', 'Taylor Decker']

ol = run_lineset[run_lineset['displayName'].isin(ol_players)]
o_avg_run = ol.groupby('displayName')['o'].mean().reset_index()
o_avg_run.rename(columns={'o': 'avg_o'}, inplace=True)
o_avg_run

,displayName,avg_o
0,Dan Skipper,86.090000
1,Evan Brown,107.011818
2,Frank Ragnow,89.655455
3,Penei Sewell,95.530909
4,Taylor Decker,90.299091


In [8]:
pass_lineset = pd.DataFrame()

for player in pass_sb['nflId'].unique().tolist()[:-1]:
    df_tobeadded = pass_sb[pass_sb['nflId'] == player]
    lineset_index = df_tobeadded.index[df_tobeadded['event'] == 'line_set'].tolist()
    lineset_index = lineset_index[0]
    ballsnap_index = df_tobeadded.index[df_tobeadded["event"] == "ball_snap"].tolist()
    ballsnap_index = ballsnap_index[0]

    df_tobeadded = df_tobeadded.loc[lineset_index:ballsnap_index]

    pass_lineset = pd.concat([pass_lineset, df_tobeadded])

ol = pass_lineset[pass_lineset['displayName'].isin(ol_players)]
o_avg_pass = ol.groupby('displayName')['o'].mean().reset_index()
o_avg_pass.rename(columns={'o': 'avg_o'}, inplace=True)
o_avg_pass

,displayName,avg_o
0,Dan Skipper,94.939206
1,Evan Brown,78.167619
2,Frank Ragnow,82.293810
3,Penei Sewell,82.354921
4,Taylor Decker,70.731429


### <center>Now we need to expound this for the whole using data for singleback formation</center>

In [9]:
sb_runs = detroit_plays[(detroit_plays['offenseFormation'] == 'SINGLEBACK') & (detroit_plays['passResult'].isna())]
sb_passes = detroit_plays[(detroit_plays['offenseFormation'] == 'SINGLEBACK') & (detroit_plays['passResult'].notna())]

sb_runs_playids = sb_runs['playId'].tolist()
sb_passes_playids = sb_passes['playId'].tolist()

runs_raw = pd.DataFrame()
runs = pd.DataFrame()
for play in sb_runs_playids:
    tobeadded = week4[(week4['gameId'] == 2022100205) & (week4['playId'] == play)]
    runs_raw = pd.concat([runs_raw, tobeadded])
    players = [x for x in runs_raw['nflId'].unique().tolist() if x == x]
    
    for player in players:
        tobeadded = runs_raw[runs_raw['nflId'] == player]
        lineset_index = tobeadded.index[tobeadded['event'] == 'line_set'].tolist()
        lineset_index = lineset_index[0]
        ballsnap_index = tobeadded.index[tobeadded["event"] == "ball_snap"].tolist()
        ballsnap_index = ballsnap_index[0]

        tobeadded = tobeadded.loc[lineset_index:ballsnap_index]

        runs = pd.concat([runs, tobeadded])

runs[runs['club'] == 'DET']['playId'].unique()

array([2314, 2625, 1084, 2648, 2356])

In [10]:
sb_runs_playids

[2314, 2625, 1084, 1133, 2648, 443, 1497, 2356, 1547, 1793, 1939]

In [11]:
runs_raw['playId'].unique()

array([2314, 2625, 1084, 1133, 2648,  443, 1497, 2356, 1547, 1793, 1939])

In [12]:
week4[(week4['gameId'] == 2022100205) & (week4['playId'] == 1133) & (week4['event'] == 'line_set')]

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
4212484,2022100205,1133,35562.0,Al Woods,35,BEFORE_SNAP,2022-10-02 17:45:20.6,99.0,SEA,right,79.000000,28.80,0.40,0.73,0.04,262.36,223.96,line_set
4212668,2022100205,1133,42543.0,Quandre Diggs,35,BEFORE_SNAP,2022-10-02 17:45:20.6,6.0,SEA,right,88.090000,25.35,1.48,0.37,0.15,253.53,245.60,line_set
4212852,2022100205,1133,43290.0,Jared Goff,35,BEFORE_SNAP,2022-10-02 17:45:20.6,16.0,DET,right,76.120000,29.90,0.06,0.83,0.01,58.47,61.62,line_set
4213036,2022100205,1133,43305.0,Taylor Decker,35,BEFORE_SNAP,2022-10-02 17:45:20.6,68.0,DET,right,76.520000,33.06,0.21,0.08,0.02,92.88,60.19,line_set
4213220,2022100205,1133,44855.0,Sidney Jones,35,BEFORE_SNAP,2022-10-02 17:45:20.6,23.0,SEA,right,83.000000,18.14,1.38,0.40,0.14,348.01,23.84,line_set
4213404,2022100205,1133,44873.0,Josh Jones,35,BEFORE_SNAP,2022-10-02 17:45:20.6,13.0,SEA,right,86.420000,34.89,1.21,0.21,0.12,210.94,159.96,line_set
4213588,2022100205,1133,44930.0,Josh Reynolds,35,BEFORE_SNAP,2022-10-02 17:45:20.6,8.0,DET,right,76.970000,38.05,0.20,0.51,0.01,101.31,302.73,line_set
4213772,2022100205,1133,44947.0,Jamaal Williams,35,BEFORE_SNAP,2022-10-02 17:45:20.6,30.0,DET,right,70.080000,29.94,0.15,0.15,0.03,88.02,103.35,line_set
4213956,2022100205,1133,45721.0,Dan Skipper,35,BEFORE_SNAP,2022-10-02 17:45:20.6,70.0,DET,right,76.540000,31.33,0.05,0.03,0.01,92.55,265.34,line_set
4214140,2022100205,1133,46089.0,Frank Ragnow,35,BEFORE_SNAP,2022-10-02 17:45:20.6,77.0,DET,right,76.880000,29.84,0.15,0.11,0.01,74.36,266.16,line_set
